In [19]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:

with open('../1_data_pre_processing/AmznLabelledNewsData.pkl', 'rb') as f:
    labelled_amzn_news_df = pickle.load(f)
    
with open('../1_data_pre_processing/AaplLabelledNewsData.pkl', 'rb') as f:
    labelled_aapl_news_df = pickle.load(f)

In [31]:
df =labelled_amzn_news_df.loc[0:50]
df

,news_timestamp,stock_timestamp,source,tokens,label
0,2017-12-15 12:04:00,2017-12-18,seekingalpha.com,"['amazon', 'soon', 'sell', 'pull', 'e', 'platf...",1
1,2017-12-21 12:04:00,2017-12-22,seekingalpha.com,"['line', 'cashier', 'similar', 'fashion', 'fut...",-1
2,2017-12-21 12:27:00,2017-12-22,thestreet.com,"['fit', 'fang', 'stock', 'amazon', 'alphabet',...",-1
3,2017-12-22 00:00:00,2017-12-26,investors.com,"['amazon', 'cloud', 'china']",1
4,2017-12-22 20:57:00,2017-12-26,yahoo.com,"['stream', 'stick', 'plus', 'offer', 'play', '...",1
5,2017-12-27 00:00:00,2017-12-28,investors.com,"['new', 'year', 'find', 'almost', 'shape', 're...",-1
6,2017-12-27 11:43:00,2017-12-28,seekingalpha.com,"['amazon', 'ad', 'product', 'platform', 'push'...",-1
7,2017-12-28 00:00:00,2017-12-29,investors.com,"['amazon', 'strong', 'holiday', 'result', 'ros...",-1
8,2017-12-28 11:39:00,2017-12-29,seekingalpha.com,"['crown', 'bin', 'look', 'give', 'kingdom', 'h...",-1
9,2017-12-28 11:39:00,2017-12-29,seekingalpha.com,"['crown', 'bin', 'look', 'give', 'kingdom', 'h...",-1


In [33]:
df['tokens'] = df['tokens'].apply(eval)
# Convert the list of tokens into space-separated strings
df['tokens_str'] = df['tokens'].apply(lambda x: ' '.join(x))

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the 'tokens_str' column
tfidf_matrix = tfidf_vectorizer.fit_transform(df['tokens_str'])

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())


/tmp/ipykernel_1318223/3314144266.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = df['tokens'].apply(eval)
/tmp/ipykernel_1318223/3314144266.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens_str'] = df['tokens'].apply(lambda x: ' '.join(x))


In [30]:
# Concatenate the original DataFrame with the TF-IDF DataFrame
df = pd.concat([df, tfidf_df], axis=1)
df

,news_timestamp,stock_timestamp,source,tokens,label,tokens_str,access,accord,acker,acronym,...,video,view,viewer,virtual,vision,vol,year,yes,yet,york
0,2017-12-15 12:04:00,2017-12-18,seekingalpha.com,"[amazon, soon, sell, pull, e, platform, two, y...",1,amazon soon sell pull e platform two year ago ...,0.000000,0.000000,0.000000,0.000000,...,0.288490,0.000000,0.000000,0.000000,0.000000,0.000000,0.204825,0.000000,0.000000,0.000000
1,2017-12-21 12:04:00,2017-12-22,seekingalpha.com,"[line, cashier, similar, fashion, futurist, am...",-1,line cashier similar fashion futurist amazon g...,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.174867,0.000000,0.331937,0.000000
2,2017-12-21 12:27:00,2017-12-22,thestreet.com,"[fit, fang, stock, amazon, alphabet, drug, int...",-1,fit fang stock amazon alphabet drug intern,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2017-12-22 00:00:00,2017-12-26,investors.com,"[amazon, cloud, china]",1,amazon cloud china,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2017-12-22 20:57:00,2017-12-26,yahoo.com,"[stream, stick, plus, offer, play, amazon, con...",1,stream stick plus offer play amazon content ge...,0.151986,0.000000,0.000000,0.000000,...,0.112773,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,2017-12-27 00:00:00,2017-12-28,investors.com,"[new, year, find, almost, shape, read, stock, ...",-1,new year find almost shape read stock cap big ...,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.176836,0.000000,0.000000,0.000000,0.000000,0.118701,0.000000,0.000000,0.000000
6,2017-12-27 11:43:00,2017-12-28,seekingalpha.com,"[amazon, ad, product, platform, push, could, t...",-1,amazon ad product platform push could target e...,0.000000,0.000000,0.000000,0.000000,...,0.152234,0.000000,0.000000,0.000000,0.000000,0.000000,0.108084,0.000000,0.000000,0.000000
7,2017-12-28 00:00:00,2017-12-29,investors.com,"[amazon, strong, holiday, result, rose, report...",-1,amazon strong holiday result rose report said ...,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,2017-12-28 11:39:00,2017-12-29,seekingalpha.com,"[crown, bin, look, give, kingdom, high, tech, ...",-1,crown bin look give kingdom high tech amazon d...,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,2017-12-28 11:39:00,2017-12-29,seekingalpha.com,"[crown, bin, look, give, kingdom, high, tech, ...",-1,crown bin look give kingdom high tech amazon d...,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
